- Total number of stops per route
- Planned total distance per route (sum of DistanceP)
- Actual total distance per route (sum of DistanceA)
- Distance deviation per route (actual total distance minus planned total distance)
- Distance efficiency ratio per route (actual total distance divided by planned total distance)
- Number of sequence deviations per route (count of stops where IndexP is not equal to IndexA)
- Deviation rate per route (sequence deviations divided by total stops)
- Number of SLA violations per route (count of stops where Arrived Time is greater than Latest Time)
- SLA violation rate per route (SLA violations divided by total stops)
- Total route duration per route (maximum Arrived Time minus minimum Arrived Time)


In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv(r'D:\Coding\last-mile-route-deviation-analytics\data\raw\routes_performance.csv')

In [4]:
df.head()

,Route ID,Driver ID,Stop ID,Address ID,Week ID,Country,Day of Week,IndexP,IndexA,Arrived Time,Earliest Time,Latest Time,DistanceP,DistanceA,Depot,Delivery
0,0,0,0,0,0,1,Monday,0,0,42.275,0.0,360.0,0.000000,0.000000,1,0
1,0,0,1,1,0,1,Tuesday,1,4,332.788,240.0,480.0,16.329053,16.329053,0,1
2,0,0,2,2,0,1,Tuesday,2,5,332.956,120.0,540.0,0.373110,0.373110,0,1
3,0,0,3,3,0,1,Monday,3,2,244.994,60.0,540.0,2.491915,0.000000,0,1
4,0,0,4,3,0,1,Monday,4,1,244.855,60.0,540.0,0.000000,13.944962,0,1


In [8]:
df['sla_voilation']=df['Arrived Time']>df['Latest Time']
df["sequence_deviation"] = df["IndexP"] != df["IndexA"]

In [13]:
route_metrics = df.groupby('Route ID').agg(
    total_stops=('Route ID','count'),
    planned_distance =('DistanceP','max'),
    actual_distance =('DistanceA','max'),
    deviation_count=('sequence_deviation','sum'),
    sla_voilation_count=('sla_voilation','sum'),
    route_start_time =('Arrived Time','min'),
    route_end_time =('Arrived Time','max')
).reset_index()

route_metrics['distance_deviation'] = route_metrics['actual_distance'] - route_metrics['planned_distance']
route_metrics['distance_efficiency_ratio']= route_metrics['actual_distance'] / route_metrics['planned_distance']
route_metrics['route_duration']= route_metrics['route_end_time'] - route_metrics['route_start_time']
route_metrics['sla_voilation_rate'] = route_metrics['sla_voilation_count'] / route_metrics['total_stops']
route_metrics['deviation_rate'] = route_metrics['deviation_count'] / route_metrics['total_stops']

In [16]:
route_metrics.head()

,Route ID,total_stops,planned_distance,actual_distance,deviation_count,sla_voilation_count,route_start_time,route_end_time,distance_deviation,distance_efficiency_ratio,route_duration,sla_voilation_rate,deviation_rate
0,0,7,16.329053,16.329053,5,0,42.275,373.553,0.000000,1.000000,331.278,0.0,0.714286
1,1,7,16.767937,12.767082,6,0,64.855,371.387,-4.000855,0.761399,306.532,0.0,0.857143
2,2,7,8.223312,9.606229,4,0,110.283,473.273,1.382918,1.168170,362.990,0.0,0.571429
3,3,10,14.694940,14.694940,6,0,194.274,448.366,0.000000,1.000000,254.092,0.0,0.600000
4,4,8,10.358857,10.358857,2,0,196.588,357.340,0.000000,1.000000,160.752,0.0,0.250000
